In [ ]:
!pip install kaggle

In [ ]:
!pip install rouge

In [ ]:
import pandas as pd
import numpy as np
from rouge import Rouge

In [ ]:
rouge = Rouge()

In [ ]:
# configuring the path of Kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d gowrishankarp/newspaper-text-summarization-cnn-dailymail

Dataset URL: https://www.kaggle.com/datasets/gowrishankarp/newspaper-text-summarization-cnn-dailymail
License(s): CC0-1.0
100% 503M/503M [00:21<00:00, 24.7MB/s]
100% 503M/503M [00:21<00:00, 24.7MB/s]


In [ ]:
from zipfile import ZipFile
dataset = '/content/newspaper-text-summarization-cnn-dailymail.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


In [ ]:
df_train=pd.read_csv('/content/cnn_dailymail/train.csv')
df_test=pd.read_csv('/content/cnn_dailymail/test.csv')
df_val=pd.read_csv('/content/cnn_dailymail/validation.csv')

In [ ]:
print(df_train.shape)
print(df_test.shape)
print(df_val.shape)

(287113, 3)
(11490, 3)
(13368, 3)


In [ ]:
df_train.describe()

,id,article,highlights
count,287113,287113,287113
unique,287113,284005,282197
top,0001d1afc246a7964130f43ae940af6bc6c57f01,(CNN) -- Dubai could lose its place on the Wom...,This page includes the show Transcript and the...
freq,1,3,83


In [ ]:
df_test.describe()

,id,article,highlights
count,11490,11490,11490
unique,11490,11488,11449
top,92c514c913c0bdfe25341af9fd72b29db544099b,Defiant Nigel Farage today insisted he did not...,Ukip leader Nigel Farage risks alienating thos...
freq,1,2,3


In [ ]:
df_val.describe()

,id,article,highlights
count,13368,13368,13368
unique,13368,13368,13300
top,61df4979ac5fcc2b71be46ed6fe5a46ce7f071c3,"Sally Forrest, an actress-dancer who graced th...",This page includes the show Transcript .\nUse ...
freq,1,1,16


In [ ]:
df_train.isnull().sum()

,0
id,0
article,0
highlights,0


In [ ]:
df_test.isnull().sum()

,0
id,0
article,0
highlights,0


In [ ]:
df_val.isnull().sum()

,0
id,0
article,0
highlights,0


In [ ]:
df_train['article'][3]

"(CNN) -- With a breezy sweep of his pen President Vladimir Putin wrote a new chapter into Crimea's turbulent history, committing the region to a future returned to Russian domain. Sixty years prior, Ukraine's breakaway peninsula was signed away just as swiftly by Soviet leader Nikita Khrushchev. But dealing with such a blatant land grab on its eastern flank won't be anywhere near as quick and easy for Europe's 28-member union. Because, unlike Crimea's rushed referendum, everyone has a say. After initially slapping visa restrictions and asset freezes on a limited number of little known politicians and military men, Europe is facing urgent calls to widen the scope of its measures to target the Russian business community in particular. The logic of this is that those who run Russia and own it are essentially two sides of the coin. Alexei Navalny, one-time Moscow mayoral contender now under house arrest for opposing the current regime, called for Europe's leaders to ban everyone -- from V

In [ ]:
df_test['article'][5]

"This is the moment that a crew of firefighters struggled to haul a giant  pig out of a garden swimming pool. The prize porker, known  as Pigwig, had fallen into the pool in an upmarket neighbourhood in Ringwood, Hampshire. His owners had been taking him for a walk around the garden when the animal plunged into the water and was unable to get out. A team from Dorset Fire and Rescue struggled to haul the huge black pig out of swimming pool water . The prize porker known as Pigwig had fallen into the water and had then been unable to get out again . Two fire crews and a specialist animal rescue team had to use slide boards and strops to haul the huge black pig from the small pool. A spokesman for Dorset Fire and Rescue Service said: 'At 4.50pm yesterday the service received a call to a pig stuck in a swimming pool. 'One crew of firefighters from Ferndown and a specialist animal rescue unit from Poole were mobilised to this incident. 'Once in attendance the crew secured the pig with strop

In [ ]:
df_val['article'][10]

'An American woman who went on the run for six years rather than face jail over a fatal DUI has been linked to a second death - after allegedly treating a backpacker with an \'African potion\' at a backstreet Thai clinic. Kelly Lynn Miller, 36 - a former Alabama stripper and prostitute - was handed a five-year sentence for mowing down truck driver Donald Goodwin, 56, in 2004. Miller was released on $50,000 appeal bond in 2009 but fled to Thailand, where she changed her name to Bea, switched from blonde to brunette and threw away her U.S. passport. Brazen: Kelly Lynn Miller, a 36-year-old fugitive, was pictured topping up her lip gloss at a press conference after being hauled in front of the media following her six-year flight from justice in Thailand . Victim: Brodie Smith went to Thailand with his girlfreind Kara Spark to attempt to kick drugs. He died after taking ibogaine at a \'clinic\' run by Victor Cracknell and Mike Picone, his mother and girlfriend said. The fugitive, who gave 

In [ ]:
sample_text = df_train['article'][3]
sample_summary = df_train['highlights'][3]
sample_text

"(CNN) -- With a breezy sweep of his pen President Vladimir Putin wrote a new chapter into Crimea's turbulent history, committing the region to a future returned to Russian domain. Sixty years prior, Ukraine's breakaway peninsula was signed away just as swiftly by Soviet leader Nikita Khrushchev. But dealing with such a blatant land grab on its eastern flank won't be anywhere near as quick and easy for Europe's 28-member union. Because, unlike Crimea's rushed referendum, everyone has a say. After initially slapping visa restrictions and asset freezes on a limited number of little known politicians and military men, Europe is facing urgent calls to widen the scope of its measures to target the Russian business community in particular. The logic of this is that those who run Russia and own it are essentially two sides of the coin. Alexei Navalny, one-time Moscow mayoral contender now under house arrest for opposing the current regime, called for Europe's leaders to ban everyone -- from V

In [ ]:
sample_summary

"Nina dos Santos says Europe must be ready to accept sanctions will hurt both sides .\nTargeting Russia's business community would be one way of sapping their support for President Putin, she says .\nBut she says Europe would have a hard time keeping its factories going without power from the east ."

# Pegasus Model

In [ ]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration

model_name = 'google/pegasus-cnn_dailymail'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [ ]:
def summarize_pegasus(text):
    inputs = tokenizer(text, max_length=1024, truncation=True, return_tensors="pt")
    summary_ids = model.generate(inputs["input_ids"], num_beams=4, max_length=200, min_length=50, length_penalty=2.0, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [ ]:
pegasus_summary = summarize_pegasus(sample_text)
pegasus_summary

"The EU is facing urgent calls to widen the scope of its measures to target the Russian business community.<n>Alexei Navalny, one-time Moscow mayoral contender now under house arrest for opposing the current regime, called for Europe's leaders to ban everyone.<n>Targeting Russia's peripatetic business community would be one way of sapping their tenuous support for President Putin.<n>But Europe would have a hard time keeping its factories going and citizens warm without power from the east."

# Calculating Rouge Scores for Pegasus

In [ ]:
scores = rouge.get_scores([sample_summary], [pegasus_summary], avg=True)

# Print the ROUGE scores
print(f"ROUGE-1: {scores['rouge-1']['f']:.4f}")
print(f"ROUGE-2: {scores['rouge-2']['f']:.4f}")
print(f"ROUGE-L: {scores['rouge-l']['f']:.4f}")

ROUGE-1: 0.5138
ROUGE-2: 0.3770
ROUGE-L: 0.5138


In [ ]:
summaries = []
references = []

for article in df_train['article'][:1000]:
    summary = summarize_pegasus(article)
    summaries.append(summary)

for highlight in df_train['highlights'][:1000]:
    references.append(highlight)

scores = rouge.get_scores(summaries, references, avg=True)

print(f"ROUGE-1: {scores['rouge-1']['f']:.4f}")
print(f"ROUGE-2: {scores['rouge-2']['f']:.4f}")
print(f"ROUGE-L: {scores['rouge-l']['f']:.4f}")


ROUGE-1: 0.4433
ROUGE-2: 0.2500
ROUGE-L: 0.4253
